# import libraries

In [4]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import scipy.sparse as sparse
import random

# import preprocessing codes
%cd /home/user_3/medistream-recsys/Script
from preprocessing import drop_columns,dict_to_column,dict_to_set,set_to_column,key_to_element

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

# ignore warning 
import warnings
warnings.filterwarnings("ignore")

/home/user_3/medistream-recsys/Script


# Upload data

In [2]:
df1 = pd.read_json('/fastcampus-data/select_column_version_3.json')
df = df1.copy()

In [20]:
# log data의 기간(4개월)과 동일한 기간의 데이터 사용하기 위해 start date 지정 
start_date = "2022-05-12"

# paid orders만 가져오기
df['date_paid'] = pd.to_datetime(df['date_paid'])
df_only_paid = df[~df['date_paid'].isna()]

# 3개월치 데이터만 가져오기
df_date = df_only_paid[df_only_paid['date_paid'] > start_date]

# 취소 안된 것만 가져오기
complete_df = df_date[(df_date['paid'] == True) & (df_date['cancelled']==False)]
complete_df = complete_df.reset_index(drop=True)


# 이상치 제거: 국시특강, 보수교육
complete_df = complete_df[complete_df['name_x'].str.contains('국시특강') == False]
complete_df = complete_df[complete_df['name_x'].str.contains('보수교육') == False]
complete_df = complete_df.sort_values(by='date_paid')


# train test split

In [21]:
from datetime import datetime, timedelta

# 마지막 일주일을 test data로 사용
date = complete_df['date_paid'].max()-timedelta(weeks=1)
train_before_preprocess = complete_df[complete_df['date_paid'] < date]
test_before_preprocess = complete_df[complete_df['date_paid'] >= date]

# train 1개 이상 구매 기록유저에 대한 학습 진행
train_drop_row_index = train_before_preprocess['customer_id'].value_counts()[train_before_preprocess['customer_id'].value_counts()>=1].index
train = train_before_preprocess[train_before_preprocess['customer_id'].isin(train_drop_row_index)]

In [22]:
# 전처리할 test index 확인: 2번 이상 구매한 고객만 추출
drop_row_index = test_before_preprocess['customer_id'].value_counts()[test_before_preprocess['customer_id'].value_counts()>2].index

# 최근 구매 3개 기록을 test로 사용
test_list = []
for idx in drop_row_index:
    test_list.append(test_before_preprocess[test_before_preprocess['customer_id']==idx].iloc[:])
test = pd.concat(test_list)

# train 있는 유저에 대해서만 진행
test = test[test['customer_id'].isin(train['customer_id'])]

In [23]:
train.date_paid = pd.to_datetime(train.date_paid)

In [24]:
# date_paid에 날짜만 남기기
train.date_paid = train.date_paid.apply(lambda x: x.date)

# train test eda

### 전처리 전후 비교

In [25]:
print('train 전처리 전:',len(train_before_preprocess), 'train 전처리 후:',len(train))

train 전처리 전: 27806 train 전처리 후: 27806


In [26]:
print('test 전처리 전:',len(test_before_preprocess), 'test 전처리 후:',len(test))

test 전처리 전: 1443 test 전처리 후: 629


### user 수 비교

In [27]:
print('train 유저 수:',len(train.customer_id.unique()))

train 유저 수: 4844


In [28]:
print('test 유저 수:',len(test.customer_id.unique()))

test 유저 수: 103


### item 개수 비교

In [29]:
print('train 아이템 수 :',len(set(train.product_ids)), 'test 아이템 수 :',len(set(test.product_ids)))

train 아이템 수 : 1915 test 아이템 수 : 269


In [30]:
print('train 만 있는 아이템 수:',  len(set(train.product_ids)-set(test.product_ids)))

train 만 있는 아이템 수: 1657


In [31]:
print('test 만 있는 아이템 수:', len(set(test.product_ids) - set(train.product_ids)))

test 만 있는 아이템 수: 11


In [32]:
print(train.shape)
print(test.shape)


(27806, 16)
(629, 16)


# Load Transactions, Reduce Memory

In [33]:
train = train[['date_paid','customer_id','name_x','product_ids']]

In [34]:
tmp = train.groupby('customer_id').date_paid.max().reset_index()
tmp.columns = ['customer_id','max_dat']
train = train.merge(tmp,on=['customer_id'],how='left')
train['diff_dat'] = (train.max_dat - train.date_paid).dt.days

# 다른 날짜를 6보다 작은 것으로 하는건 최신의 정보를 반영하기 위함인가?
train = train.loc[train['diff_dat']<=6]
print('Train shape:',train.shape)

Train shape: (8520, 6)


In [35]:
train.diff_dat.unique()

array([0, 4, 1, 5, 3, 2, 6])

# Apriori Transformation:

In [36]:
#Apriori treats duplicates as a single occurence
# 한 유저당 구매한 제품들의 unique -> 중복을 없앰.
train_apri = train.groupby('customer_id')['name_x'].unique().reset_index()
train_apri

,customer_id,name_x
0,5d60b9184e77525ec5ca138d,"[[2022전국한의학학술대회] MET에 근거한 경근추나, [2022전국한의학학술대회..."
1,5d60b93c4e77525ec5ca1391,"[복순도가 손막걸리 3병 고급선물세트, Freshvalley 추석 프리미엄 청도 ..."
2,5d60c43f4e77525ec5ca13a8,"[[타스컴] 백톤디킨슨(BD) 인슐린주사기 100pcs, 진맛과 명인 특선 16호 ..."
3,5d60c5884e77525ec5ca13ae,"[맘 편하게 한약 처방하기, 선우항 선생님의 청구 원포인트 레슨, 신민섭 교수님의 ..."
4,5d60cab24e77525ec5ca13d5,[숲을 보는 요통치료]
...,...,...
4839,630871b668554000236d8b03,[2022년 사상체질의학회 제1회 월례학술집담회 녹화본 강의(2)]
4840,630eaa6a68554000236da5d9,[영어 진료 가이드북 ]
4841,630f06ee974f2b0022f85afa,[아큐비즈 포켓 Acuviz Pocket 초음파]
4842,63155514c640eb0021af84fc,[임상 한의사를 위한 기본 한약처방 강의 2판]


In [37]:
from mlxtend.preprocessing import TransactionEncoder

In [38]:
pd.set_option('display.max_colwidth', -1)
transactions = [a[1]['name_x'].tolist() for a in list(train.groupby(['customer_id','date_paid']))]

In [39]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

# orders_1hot

In [40]:
orders_1hot = pd.DataFrame(te_ary, columns=te.columns_)
pf = orders_1hot.describe()

In [41]:
f = pf.iloc[0]-pf.iloc[3] # count - frequent
a = f.tolist()
b = list(f.index)
item = pd.DataFrame([[a[r],b[r]]for r in range(len(a))], columns=['Count','Item'])
item = item.sort_values(['Count'], ascending=False).head(10)
print("MOST POPULAR")
item

MOST POPULAR


,Count,Item
1134,273,초음파 유도하 침 시술 가이드북
757,264,비만문답
1226,223,플로차트 정형외과 진단
730,223,방성혜 부인과 전문의서 처방 강의
749,205,"부항컵 동방 일회용 1,000개"
731,187,방성혜 피부과 전문의서 처방 강의
32,175,<도전! 베스트 강의> 올인원 패키지
996,152,윤상훈·권병조의 알짜 근육학
958,146,영어 진료 가이드북
722,110,미코 바이오메드 VERI-Q 코로나 신속항원진단키트 전문가용 (20T)


In [42]:
from mlxtend.frequent_patterns import apriori

In [43]:
is_ap = apriori(orders_1hot, min_support=0.01, max_len=3, use_colnames=True)

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
is_ap.sort_values(by = ['support'], ascending = False).head(15)

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
12,0.047561,(초음파 유도하 침 시술 가이드북)
7,0.045993,(비만문답)
13,0.038850,(플로차트 정형외과 진단)
4,0.038850,(방성혜 부인과 전문의서 처방 강의)
6,0.035714,"(부항컵 동방 일회용 1,000개)"
5,0.032578,(방성혜 피부과 전문의서 처방 강의)
1,0.030488,(<도전! 베스트 강의> 올인원 패키지)
11,0.026481,(윤상훈·권병조의 알짜 근육학)
9,0.025436,(영어 진료 가이드북 )
3,0.019164,(미코 바이오메드 VERI-Q 코로나 신속항원진단키트 전문가용 (20T))


# Calculate Association Rules

In [45]:
from mlxtend.frequent_patterns import association_rules

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [46]:
rules_confidence = association_rules(is_ap, min_threshold=0.01) # default가 confidence
rules_lift = association_rules(is_ap, metric= "lift", min_threshold=0.01)
display(rules_lift.tail())
# display(rules_confidence)

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(방성혜 부인과 전문의서 처방 강의),(방성혜 피부과 전문의서 처방 강의),0.038850,0.032578,0.018467,0.475336,14.590537,0.017201,1.843889
1,(방성혜 피부과 전문의서 처방 강의),(방성혜 부인과 전문의서 처방 강의),0.032578,0.038850,0.018467,0.566845,14.590537,0.017201,2.218951


In [47]:
rules_confidence_item = association_rules(is_ap, min) # default가 confidence
# rules_lift = association_rules(is_ap, metric= "lift", min_threshold=0.01)
# display(rules_lift)
display(rules_confidence_item)

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ValueError: Metric must be 'confidence' or 'lift', got '<built-in function min>'

In [48]:
display(rules_lift.sort_values(by=['support'], ascending = False).head(10))
display(rules_confidence.sort_values(by=['support'], ascending = False).head(10))


/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(방성혜 부인과 전문의서 처방 강의),(방성혜 피부과 전문의서 처방 강의),0.038850,0.032578,0.018467,0.475336,14.590537,0.017201,1.843889
1,(방성혜 피부과 전문의서 처방 강의),(방성혜 부인과 전문의서 처방 강의),0.032578,0.038850,0.018467,0.566845,14.590537,0.017201,2.218951


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(방성혜 부인과 전문의서 처방 강의),(방성혜 피부과 전문의서 처방 강의),0.038850,0.032578,0.018467,0.475336,14.590537,0.017201,1.843889
1,(방성혜 피부과 전문의서 처방 강의),(방성혜 부인과 전문의서 처방 강의),0.032578,0.038850,0.018467,0.566845,14.590537,0.017201,2.218951


In [49]:
#  we can sort the association rules according to leverage value and find the most positively correlated item sets.
display(rules_lift.sort_values(by=['conviction'], ascending = False))

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(방성혜 피부과 전문의서 처방 강의),(방성혜 부인과 전문의서 처방 강의),0.032578,0.038850,0.018467,0.566845,14.590537,0.017201,2.218951
0,(방성혜 부인과 전문의서 처방 강의),(방성혜 피부과 전문의서 처방 강의),0.038850,0.032578,0.018467,0.475336,14.590537,0.017201,1.843889


# (1) Recommend Most Often Previously Purchased Items

In [60]:
tmp = train.groupby(['customer_id','name_x'])['date_paid'].agg('count').reset_index()
tmp.columns = ['customer_id','name_x','ct']
train = train.merge(tmp,on=['customer_id','name_x'],how='left')
train = train.sort_values(['ct','date_paid'],ascending=False)
train = train.drop_duplicates(['customer_id','name_x'])
train = train.sort_values(['ct', 'name_x'],ascending=False)
train

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date_paid,customer_id,name_x,product_ids,max_dat,diff_dat,ct_x,ct_y,ct
375,2022-09-05,5e53b0f1bfe42609447827e4,흔히보는 정형외과 외래진료 가이드북,6182113bbc63410023473754,2022-09-05,0,1,1,1
376,2022-08-30,5db15bc40dabe405b156f326,흔히보는 정형외과 외래진료 가이드북,6182113bbc63410023473754,2022-09-03,4,1,1,1
377,2022-08-29,5e66dc73bfe42609447832fd,흔히보는 정형외과 외래진료 가이드북,6182113bbc63410023473754,2022-08-29,0,1,1,1
378,2022-08-28,60a5abfdc1908b001a64756f,흔히보는 정형외과 외래진료 가이드북,6182113bbc63410023473754,2022-08-28,0,1,1,1
379,2022-08-25,6079c3dfc1908b001a63e79f,흔히보는 정형외과 외래진료 가이드북,6182113bbc63410023473754,2022-08-25,0,1,1,1
...,...,...,...,...,...,...,...,...,...
7885,2022-08-05,61756b0c4cf9a5002288218a,"동방 수침 10,000쌈 (100,000pcs)",62b1318208e04900234e09ca,2022-08-05,0,1,1,1
374,2022-06-27,5ef6cc74334af8196b13c0bf,"동방 수침 10,000쌈 (100,000pcs)",62b1318208e04900234e09ca,2022-06-27,0,2,1,1
7886,2022-06-24,5dd6277c2bb59605ca3d055d,"동방 수침 10,000쌈 (100,000pcs)",62b1318208e04900234e09ca,2022-06-27,3,1,1,1
7887,2022-06-22,5d9a82d70dabe405b156de9b,"동방 수침 10,000쌈 (100,000pcs)",62b1318208e04900234e09ca,2022-06-22,0,1,1,1


# (2) Recommend Items Purchased Together

In [51]:
test[['customer_id','name_x','product_ids']]

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,customer_id,name_x,product_ids
56544,624e46e0d6b3320023ddcccc,메디TV Ep.39,62d0ddac53948a002289824d
56545,624e46e0d6b3320023ddcccc,메디TV Ep.39,62d0ddac53948a002289824d
56546,624e46e0d6b3320023ddcccc,메디TV Ep.39,62d0ddac53948a002289824d
56547,624e46e0d6b3320023ddcccc,메디TV Ep.39,62d0ddac53948a002289824d
56830,624e46e0d6b3320023ddcccc,메디TV Ep.39,62d0ddac53948a002289824d
...,...,...,...
56820,5d66073d4e77525ec5ca171d,메디TV Ep.32,62bbf28608e04900234e370c
57329,5d66073d4e77525ec5ca171d,메디TV Ep.31,62c2924288c7dc00239bc031
56937,5d96c1700dabe405b156db52,아크메시 사이드라인 스트레이트 엘라스틱 스크럽 남성 팬츠 DARK GRAY,5ec680fae510ee503167b2ec
56935,5d96c1700dabe405b156db52,"다나침 두쌈용 1,000쌈 (10,000pcs)",5fe3ffec329d0c001aca7296


In [52]:
is_ap = apriori(orders_1hot, min_support=0.01, use_colnames=True)
rules_confidence_item = association_rules(is_ap, min_threshold=0.5) # default가 confidence
rules_confidence_item

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(방성혜 피부과 전문의서 처방 강의),(방성혜 부인과 전문의서 처방 강의),0.032578,0.03885,0.018467,0.566845,14.590537,0.017201,2.218951


In [53]:
apriori_items = rules_confidence_item[['antecedents','consequents']].iloc[:15]

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [54]:
apriori_items['antecedents'] = apriori_items['antecedents'].apply(lambda x: list(map(str,x))[0])
apriori_items['consequents'] = apriori_items['consequents'].apply(lambda x: list(map(str,x))[0])

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [55]:
apriori_items

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents
0,방성혜 피부과 전문의서 처방 강의,방성혜 부인과 전문의서 처방 강의


In [56]:
train_items = train[['date_paid','customer_id','name_x','product_ids']]

# apriori_items
# train_items[train_items['name_x']==]
train_items['apriori_pred'] = train_items['name_x'].apply(lambda x: apriori_items[apriori_items['antecedents']==x]['consequents'].values[0] if x in list(apriori_items['antecedents']) \
                            else None)
# for idx, row in train_items.iterrows():

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [57]:
name_x_df = train_items.groupby('customer_id')['name_x'].unique().reset_index()
apriori_pred_df  = train_items.groupby('customer_id')['apriori_pred'].unique().reset_index()
real_pred = test.groupby('customer_id')['name_x'].unique().reset_index().rename(columns={'name_x':'real_product'})

train_pred = pd.merge(name_x_df,apriori_pred_df, on='customer_id', how='left')
train_real_pred = pd.merge(train_pred,real_pred, on='customer_id', how='right')

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [58]:
train_real_pred

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,customer_id,name_x,apriori_pred,real_product
0,5d60cc294e77525ec5ca13e0,"[추가 정산 (과세), 온뜸 의료용 양면테이프, 영어 진료 가이드북 , 아큐비즈 포켓 Acuviz Pocket 초음파, 세이프란 전용 란셋 니들 26g (100pcs) 1box, 부항컵 동방 일회용 1,000개, 복순도가 손막걸리 3병 고급선물세트, 메디TV Ep.45, 댄싱사이더 애플사이더 다이닝에디션 세트 B, [씨케이주식회사] 지실 600g, 중국, [씨케이주식회사] 복령 (5묶음) 600g*5ea, 중국, [단군한우] 1++ 꽃등심/꽃등심/국거리 선물세트 1호, Freshvalley 추석 실속형 샤인머스캣 선물세트]",[None],"[메디TV Ep.45, [씨케이주식회사] 오미자(규) (5묶음) 600g*5ea, 중국, 안동농협 생강 500g, [메디스트림 스탠다드] 당귀 500g, 한국 5묶음]"
1,5d60cd634e77525ec5ca1402,"[메디TV Ep.31 , 메디TV Ep.28, 메디TV Ep.27 , 메디TV Ep.26 , 메디TV Ep.25 ]",[None],"[아큐비즈 포켓 Acuviz Pocket 초음파, 영어 진료 가이드북 , 외래에서 꼭 알아야 할 통증증후군 137가지, [멤버스] Medistream MEMBERS _구독가입형]"
2,5d60d0a94e77525ec5ca1446,"[메디TV Ep.34 , 메디TV Ep.33 , 메디TV Ep.43, 메디TV Ep.42, 메디TV Ep.41, 메디TV Ep.40, 메디TV Ep.39, 메디TV Ep.38, 메디TV Ep.37, 메디TV Ep.35]",[None],"[메디TV Ep.44, 메디TV Ep.45, 메디TV Ep.46, 메디TV Ep.47, 메디TV Ep.48, 메디TV Ep.01 , 메디TV Ep.02 , 메디TV Ep.03 , 메디TV Ep.04 , 메디TV Ep.05, 맘 편하게 한약 처방하기, 메디TV Ep.06 , [멤버스] Medistream MEMBERS _구독가입형]"
3,5d60f53f4e77525ec5ca14e1,"[메디TV Ep.01 , 메디TV Ep.34 , 메디TV Ep.32 , 메디TV Ep.06 ]",[None],"[메디TV Ep.06 , 메디TV Ep.10 , 메디TV Ep.09 , 메디TV Ep.34 , 메디TV Ep.05, 맘 편하게 한약 처방하기]"
4,5d6293ad4e77525ec5ca15ba,"[메디TV Ep.46, 메디TV Ep.39, [단군한우] 1++ 꽃등심/안심/채끝등심 선물세트 2호]",[None],"[메디TV Ep.40, 메디TV Ep.46]"
5,5d64e80d4e77525ec5ca16e9,"[제일 공진단 현탁액 50mL 10개입, 우전 도침 10팩(1,000pcs), 동방 정안침 1,000쌈 (10,000pcs), 대신코스코 에프론 시트페이퍼 47*36(cm) 100매 (25개), 다나침 두쌈용 10,000쌈 (100,000pcs)]",[None],"[맘 편하게 한약 처방하기, 제일 공진단 현탁액 50mL 10개입, [멤버스] Medistream MEMBERS _구독가입형]"
6,5d66073d4e77525ec5ca171d,"[메디TV Ep.32 , 메디TV Ep.31 ]",[None],"[메디TV Ep.01 , 메디TV Ep.32 , 메디TV Ep.31 ]"
7,5d6608b34e77525ec5ca171e,"[동방 스프링침 한쌈용 3,000쌈 (30,000pcs)]",[None],"[현미온미 눈 찜질팩, [메디스트림 스탠다드] 시호 [에이] 500g, 중국, [메디스트림 스탠다드] 당귀 500g, 한국, [메디스트림 스탠다드] 맥문동 500g, 중국, [메디스트림 스탠다드] 백출 [특] 500g, 중국, [메디스트림 스탠다드] 마황 500g, 중국, [메디스트림 스탠다드] 복령 500g, 중국, 부항컵 동방 일회용 100개]"
8,5d6644194e77525ec5ca1761,"[동방 필젯 인슐린주사기 (100pcs), 그린 크린워터(정제수) 18L (1개)]",[None],"[[메디스트림 스탠다드] 육계[거피] 500g, 베트남 5묶음, 함소아 심적환 150환*6병(1갑), [멤버스] Medistream MEMBERS _구독가입형]"
9,5d69cd4319efa30eb2913de3,"[Freshvalley 추석 실속형 샤인머스캣 선물세트, 압구정 현대백화점 한우 소담 송 (냉장/등심로스,국거리,불고기), 압구정 현대백화점 지정목장 와규 육포 세트, 동방 도침 (100pcs), 굿플 에어원 전동부항 흡입기 5개 묶음]",[None],"[소아피부질환해설, [멤버스] Medistream MEMBERS _구독가입형, 맘 편하게 한약 처방하기]"


In [100]:
# 아이템 맞춘 개수
cnt = 0
for preds, reals in zip(train_real_pred['apriori_pred'], train_real_pred['real_product']):
    for pred in preds:
        if pred in reals:
            cnt += 1
cnt

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0